In [1]:
#####Unsupervised Truth Discovery from Heterogeneous data source#####
import numpy as np
import pandas as pd
import matplotlib.pyplot  as plt
import numpy.matlib
import scipy.linalg
import itertools
from scipy import sparse
from scipy.stats import norm
from pandas.tools.plotting import scatter_matrix
from numpy.random import *
from scipy import optimize

#np.random.seed(98537)

In [ ]:
####データの生成####
##データの設定
M = 

In [ ]:
####データの生成####
##データの設定
k11 = 7
k21 = 8
k12 = 9
k22 = 9
item = 3000
event = 2500
Lambda = np.random.gamma(30.0, 1/0.2, item)
pt = np.random.poisson(Lambda, item)
N = np.sum(pt)
k_vec1 = np.repeat(1.0, k12)
k_vec2 = np.repeat(1.0, k22)